# NLTK Corpus Clustering with scikit-learn package

## Preparation
First of all, let us import necessary libraries.
* nltk
* sklearn


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import collections

We will use the following datasets in this tutorial.

In [2]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("semcor")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shireen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load the corpus from NLTK package and check out the contents.

The corpus I used is Semcor corpus from NLTK data. Semcor, Semantic Concordance package is a corpus provided by Princeton University which contains 352 documents from Brown corpus. It contains Brown1 (103 semantically tagged Brown Corpus files contains all content words tagged), Brown2 (83 semantically tagged Brown Corpus files contains all content words tagged), and Brownv (166 semantically tagged Brown Corpus files contains verb only).

In [3]:
from nltk.corpus import semcor as corpus

for n,item in enumerate(corpus.words(corpus.fileids()[0])[:1000]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")
#Total number of documents
print("")
print("")
print("Total number of documents:",len(corpus.fileids()))

The Fulton County Grand Jury said Friday an investigation of Atlanta 's recent primary election produced `` no evidence '' that any irregularities took place  
. The jury further said in term end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves  
the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September October term jury  
had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won  
by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the  
widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that  
many of Georgia 's registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act

In [4]:
#Train the model with first K number of documents or all documents. 
# K=352
# docs=[corpus.words(fileid) for fileid in corpus.fileids()[:K]]

# All documents
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:10])
print("num of documents:", len(docs))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Committee', 'approval', 'of', 'Gov.', 'Price', ...], ['The', 'Orioles', 'tonight', 'retained', 'the', ...], ['A', 'Texas', 'halfback', 'who', 'does', "n't", ...], ['Rookie', 'Ron', 'Nischwitz', 'continued', 'his', ...], ['Nick', 'Skorich', ',', 'the', 'line', 'coach', 'for', ...], ['If', 'the', 'Cardinals', 'heed', 'Manager', 'Gene', ...], ['Sizzling', 'temperatures', 'and', 'hot', 'summer', ...], ['The', 'nuclear', 'war', 'is', 'already', 'being', ...], ['It', 'is', 'not', 'news', 'that', 'Nathan', ...]]
num of documents: 352


## Data preprocessing
First, let us define some stopwords. Here we consider English stopwords from the NLTK package and some noises that may affect our result.  
(Optional) Try to ignore numbers and words through regular expression.

In [12]:
# English stopwords defined by the NLTK package.
en_stop = nltk.corpus.stopwords.words('english')

# Ignore noises that might affect our result.
en_stop = ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<","''"]                  \
         +en_stop

Next, let us define several preprocessing functions.

In [13]:
from nltk.corpus import wordnet as wn # import for lemmatize

def preprocess_word(word, stopwordset):
    
    #1.convert words to lowercase (e.g., Python =>python)
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",",".","'s"]:
        return None
    
    #3.remove stopwords  (e.g., the => (None)) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  (e.g., cooked=>cook)
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    # lemmatized words could be in the stopwords set
    elif lemma in stopwordset: 
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

Let us check out the preprocessing result.

In [14]:
# before
print(docs[0][:25]) 

# after
print(preprocess_documents(docs)[0][:25])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place']
['fulton', 'county', 'grand', 'jury', 'say', 'friday', 'investigation', 'atlanta', 'recent', 'primary', 'election', 'produce', 'evidence', 'irregularity', 'take', 'place', 'jury', 'say', 'term', 'end', 'presentment', 'city', 'executive', 'committee', 'over-all']


## Clustering
Document vectorization with tf-idf. We use the TfidfVectorizer that provided by the sklearn package (and set the hyperparameter).

In [15]:
# define the vectorizer
pre_docs=preprocess_documents(docs)
pre_docs=["".join(doc) for doc in pre_docs]
print(pre_docs[0])

vectorizer = TfidfVectorizer(max_features=400, token_pattern=u'(?u)\\b\\w+\\b' )

# fit
tf_idf = vectorizer.fit_transform(pre_docs)
print(tf_idf)
print("")
print("")

#Using Cosinus Similarity
def cos_similarity(pre_docs):
    tfidf = vectorizer.fit_transform(pre_docs)
    return (tfidf * tfidf.T).toarray()
cos_similarity(pre_docs)

fultoncountygrandjurysayfridayinvestigationatlantarecentprimaryelectionproduceevidenceirregularitytakeplacejurysaytermendpresentmentcityexecutivecommitteeover-allchargeelectiondeservepraisethankscityatlantamannerelectionconductseptemberoctobertermjurychargefultonsuperiorcourtjudgedurwoodpyeinvestigatereportpossibleirregularityhard-foughtprimarymayor-nominateivanallenjr.relativehandfulreportreceivejurysayconsiderwidespreadinterestelectionnumbervotersizecityjurysayfindmanygeorgiaregistrationelectionlawsoutmodeinadequateoftenambiguousrecommendfultonlegislatoractlawsstudyreviseendmodernizeimprovegrandjurycommentnumbertopicamongatlantafultoncountypurchasingdepartmentsaywelloperatefollowgenerallyacceptpracticeinurebestinterestgovernmenthoweverjurysaybelieftwoofficecombineachievegreaterefficiencyreducecostadministrationcitypurchasingdepartmentjurysaylackexperienceclericalpersonnelresultcitypersonnelpolicyurgecitytakestepsremedyproblemimplementationgeorgiaautomobiletitlelawalsorecommendoutgoju

array([[1.        , 0.22325212, 0.07018603, ..., 0.        , 0.        ,
        0.16775403],
       [0.22325212, 1.        , 0.19126537, ..., 0.28920051, 0.        ,
        0.25122288],
       [0.07018603, 0.19126537, 1.        , ..., 0.18142852, 0.        ,
        0.        ],
       ...,
       [0.        , 0.28920051, 0.18142852, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.16775403, 0.25122288, 0.        , ..., 0.        , 0.        ,
        1.        ]])

We use K-means to cluster our documents.

In [16]:
# K-means setting, K parameter = 4
num_clusters = 4
km = KMeans(n_clusters=num_clusters, random_state = 0)

# fit
clusters = km.fit_predict(tf_idf)

for doc, cls in zip(preprocess_documents(docs)[0], clusters):
    print(cls,doc)

2 fulton
2 county
2 grand
2 jury
2 say
3 friday
2 investigation
1 atlanta
2 recent
2 primary
2 election
2 produce
2 evidence
2 irregularity
2 take
2 place
2 jury
0 say
2 term
1 end
2 presentment
1 city
2 executive
2 committee
2 over-all
2 charge
3 election
2 deserve
2 praise
2 thanks
1 city
0 atlanta
0 manner
0 election
2 conduct
0 september
1 october
2 term
2 jury
2 charge
2 fulton
2 superior
2 court
0 judge
2 durwood
0 pye
2 investigate
0 report
1 possible
0 irregularity
2 hard-fought
2 primary
2 mayor-nominate
2 ivan
2 allen
2 jr.
2 relative
2 handful
2 report
2 receive
2 jury
1 say
2 consider
2 widespread
2 interest
2 election
2 number
2 voter
2 size
2 city
2 jury
2 say
2 find
2 many
2 georgia
2 registration
2 election
2 laws
1 outmode
1 inadequate
2 often
2 ambiguous
2 recommend
2 fulton
3 legislator
2 act
2 laws
2 study
3 revise
2 end
2 modernize
2 improve
2 grand
2 jury
2 comment
2 number
2 topic
2 among
2 atlanta
2 fulton
2 county
2 purchasing
2 department
2 say
3 well
2 operat

Check out the clustering result.

In [33]:
# K-means setting, K parameter = 8
num_clusters = 8
km = KMeans(n_clusters=num_clusters, random_state = 0)

# fit
clusters = km.fit_predict(tf_idf)

for doc, cls in zip(preprocess_documents(docs)[0], clusters):
    print(cls,doc)

2 fulton
6 county
2 grand
4 jury
2 say
2 friday
2 investigation
1 atlanta
7 recent
3 primary
6 election
2 produce
2 evidence
2 irregularity
4 take
2 place
2 jury
0 say
2 term
1 end
2 presentment
1 city
4 executive
6 committee
2 over-all
2 charge
2 election
2 deserve
2 praise
4 thanks
1 city
0 atlanta
0 manner
0 election
2 conduct
0 september
1 october
3 term
2 jury
2 charge
2 fulton
2 superior
2 court
0 judge
2 durwood
0 pye
2 investigate
0 report
1 possible
0 irregularity
2 hard-fought
6 primary
7 mayor-nominate
2 ivan
2 allen
4 jr.
2 relative
7 handful
2 report
2 receive
2 jury
1 say
2 consider
2 widespread
2 interest
2 election
2 number
2 voter
2 size
2 city
2 jury
2 say
2 find
2 many
2 georgia
2 registration
2 election
2 laws
3 outmode
1 inadequate
2 often
2 ambiguous
2 recommend
2 fulton
5 legislator
2 act
2 laws
2 study
2 revise
2 end
2 modernize
2 improve
2 grand
6 jury
2 comment
2 number
2 topic
2 among
2 atlanta
6 fulton
2 county
2 purchasing
2 department
4 say
2 well
4 operat

In [34]:
# K-means setting, K parameter = 9
num_clusters = 9
km = KMeans(n_clusters=num_clusters, random_state = 0)

# fit
clusters = km.fit_predict(tf_idf)

for doc, cls in zip(preprocess_documents(doｃs)[0], clusters):
    print(cls,doc)


0 fulton
0 county
0 grand
6 jury
0 say
7 friday
0 investigation
2 atlanta
0 recent
1 primary
5 election
0 produce
0 evidence
0 irregularity
6 take
0 place
0 jury
3 say
0 term
2 end
0 presentment
2 city
6 executive
0 committee
0 over-all
0 charge
7 election
0 deserve
0 praise
6 thanks
2 city
3 atlanta
3 manner
3 election
0 conduct
3 september
2 october
5 term
0 jury
5 charge
0 fulton
0 superior
0 court
3 judge
0 durwood
3 pye
0 investigate
3 report
2 possible
3 irregularity
0 hard-fought
0 primary
0 mayor-nominate
0 ivan
0 allen
6 jr.
0 relative
0 handful
0 report
0 receive
0 jury
2 say
0 consider
0 widespread
0 interest
0 election
7 number
0 voter
0 size
0 city
0 jury
0 say
0 find
0 many
0 georgia
0 registration
0 election
0 laws
1 outmode
2 inadequate
0 often
0 ambiguous
0 recommend
0 fulton
8 legislator
0 act
0 laws
0 study
0 revise
0 end
0 modernize
0 improve
0 grand
0 jury
0 comment
0 number
0 topic
0 among
0 atlanta
0 fulton
0 county
0 purchasing
5 department
6 say
7 well
6 operat